In [1]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
import datetime
import gc
from numba import cuda

In [2]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from keras import backend as K
import tensorflow as tf

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5280752032264899127
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9898950656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11067479159137294484
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:2a:00.0, compute capability: 8.6"
]


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
# Load the training data
rawtrain = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv", index_col=0)
rawtest = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv", index_col=0)

In [6]:
rawtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600000 entries, 0 to 599999
Columns: 101 entries, f0 to target
dtypes: float64(100), int64(1)
memory usage: 466.9 MB


In [7]:
rawtest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540000 entries, 600000 to 1139999
Data columns (total 100 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   f0      540000 non-null  float64
 1   f1      540000 non-null  float64
 2   f2      540000 non-null  float64
 3   f3      540000 non-null  float64
 4   f4      540000 non-null  float64
 5   f5      540000 non-null  float64
 6   f6      540000 non-null  float64
 7   f7      540000 non-null  float64
 8   f8      540000 non-null  float64
 9   f9      540000 non-null  float64
 10  f10     540000 non-null  float64
 11  f11     540000 non-null  float64
 12  f12     540000 non-null  float64
 13  f13     540000 non-null  float64
 14  f14     540000 non-null  float64
 15  f15     540000 non-null  float64
 16  f16     540000 non-null  float64
 17  f17     540000 non-null  float64
 18  f18     540000 non-null  float64
 19  f19     540000 non-null  float64
 20  f20     540000 non-null  float64
 21  f21

In [8]:
train = rawtrain.copy()
y = train['target']
features = train.drop(['target'], axis=1)

In [9]:
X = features.copy()
X.shape

(600000, 100)

In [10]:
test = rawtest.copy()
test.shape

(540000, 100)

In [11]:
lgbm_parameters = {
    "device":'gpu', "metric":"auc",
  'learning_rate': 0.08697465911179744, 'max_depth': 2, 'min_data_in_leaf': 592, 'n_estimators': 4565, 'num_leaves': 85, 'reg_alpha': 4.2395421854514055, 'reg_lambda': 3.4669998025411233
}
xgboost_parameters = { 'n_jobs':4, 
                       'tree_method': 'gpu_hist', 
                        'gpu_id': 0,
'learning_rate': 0.03237498960526522, 'n_estimators': 10837, 'gamma': 0.7820801389298118, 'max_depth': 2, 'min_child_weight': 9.149611138786955, 'subsample': 0.7488960655884316, 'lambda': 4.190627000051498, 'alpha': 6.241101997471009
                     }
catboost_parameters={
    'task_type':"GPU", "loss_function":"RMSE",
   'learning_rate': 0.04930064646089139, 'iterations': 3238, 'depth': 6, 'l2_leaf_reg': 32.60575094913488, 'bagging_temperature': 0.9250154053358728
}

In [12]:
class KerasModel:
    def __init__(self, model, session):
        self.model = model
        self.session = session
    def predict(self, test):
        out = self.model.predict(test)
        return np.reshape(out,(out.size,)) 

input_shape = [X.shape[1]]
def create_keras():
    model = keras.Sequential([
        layers.BatchNormalization(input_shape = input_shape),
        layers.Dense(units = 128, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(rate = 0.3),
        layers.Dense(units = 64, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(rate = 0.3),
        layers.Dense(units = 32, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(rate = 0.3), 
        layers.Dense(units = 1, activation = 'hard_sigmoid')
    ])
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC()]
    )
    session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
    return KerasModel(model, session)

def fit_keras(model, Xt, yt, Xv, yv):
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.0001, # minimium amount of change to count as an improvement
        patience=40, # how many epochs to wait before stopping
        restore_best_weights=True,
    )
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    history = model.model.fit(Xt, yt, validation_data=(Xv, yv), 
    callbacks = [early_stopping, tensorboard_callback], 
        batch_size=6000, epochs=300, verbose=0)
    del history
def clear_keras(model):
    gc.collect()
    del model.model
    model.session.close()
    del model.session
    gc.collect()
    K.clear_session()
    tf.compat.v1.reset_default_graph()
    gc.collect()

In [13]:
def fit_catboost(model, Xt, yt, Xv, yv):
    model.fit(Xt, yt, early_stopping_rounds = 400, eval_set=[(Xv, yv)], verbose = 0)
def fit_lgbm(model, Xt, yt, Xv, yv):
    model.fit(Xt, yt, early_stopping_rounds = 399, eval_set=[(Xv, yv)], verbose = -1)  
def fit_xgboost(model, Xt, yt, Xv, yv):
    model.fit(Xt, yt, early_stopping_rounds = 400, eval_set=[(Xv, yv)], verbose=False)
model_types = [
    {"name": "keras", "create": create_keras, "fit": fit_keras, "clear": clear_keras},
    {"name": "lgbm", "create": lambda: LGBMRegressor(**lgbm_parameters), "fit": fit_lgbm},
    #{'name': "lgbm_0", 'create': lambda: LGBMRegressor(**lgbm_parameters_0), "fit": fit_lgbm},
    #{'name': "lgbm_1", "create": lambda: LGBMRegressor(**lgbm_parameters_1), 'fit': fit_lgbm},
    #{"name": "xgboost_2", "create": lambda: XGBRegressor(**xgboost_parameters_2), "fit": fit_xgboost},
    #{"name": "xgboost_1", "create": lambda: XGBRegressor(**xgboost_parameters_1), "fit": fit_xgboost},
    {'name': 'xgboost', 'create': lambda: XGBRegressor(**xgboost_parameters, eval_metric="auc"), 'fit': fit_xgboost},
    {"name": "catboost", "create": lambda: CatBoostRegressor(**catboost_parameters), "fit": fit_catboost},
    #{'name': 'catboost_0', "create": lambda: CatBoostRegressor(**catboost_parameters_0), "fit": fit_catboost},
]

In [14]:
model_types

[{'name': 'keras',
  'create': <function __main__.create_keras()>,
  'fit': <function __main__.fit_keras(model, Xt, yt, Xv, yv)>,
  'clear': <function __main__.clear_keras(model)>},
 {'name': 'lgbm',
  'create': <function __main__.<lambda>()>,
  'fit': <function __main__.fit_lgbm(model, Xt, yt, Xv, yv)>},
 {'name': 'xgboost',
  'create': <function __main__.<lambda>()>,
  'fit': <function __main__.fit_xgboost(model, Xt, yt, Xv, yv)>},
 {'name': 'catboost',
  'create': <function __main__.<lambda>()>,
  'fit': <function __main__.fit_catboost(model, Xt, yt, Xv, yv)>}]

In [ ]:
spl = 5
out_of_fold_predictions = np.zeros((X.shape[0], len(model_types)))
meta_features = np.zeros((test.shape[0], len(model_types)))
y_pred = []
for i, model_type in enumerate(model_types):
    kf = StratifiedKFold(n_splits=spl, shuffle=True)
    mse = []
    print("starting " + model_type["name"])
    meta_features_folds = np.zeros((test.shape[0], spl))
    i_fold = 0
    for train_idx, valid_idx in tqdm(kf.split(X,y.round())):
        Xt = X.iloc[train_idx, :]
        yt = y.iloc[train_idx]
        Xv = X.iloc[valid_idx, :]
        yv = y.iloc[valid_idx]
        model = model_type['create']()
        model_type['fit'](model, Xt, yt, Xv, yv)
        y_pred = model.predict(Xv)
        out_of_fold_predictions[valid_idx, i] = y_pred
        valid_mse = roc_auc_score(y_true = yv, y_score = y_pred)
        mse.append(valid_mse)
        meta_features_folds[:, i_fold] = model.predict(test)
        print("validation quality metric", valid_mse)
        i_fold += 1
        if "clear" in model_type:
            print("clearing")
            model_type["clear"](model)
        del model
        gc.collect()
    device = cuda.get_current_device()
    device.reset()
    meta_features[:, i] = meta_features_folds.mean(axis=1)
    print(model_type["name"], i, mse, " max ", max(mse), " min ", min(mse)," avg ", sum(mse)/len(mse))

starting keras


0it [00:00, ?it/s]

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:2a:00.0, compute capability: 8.6

validation quality metric 0.7580579493340109
clearing


1it [01:56, 116.74s/it]

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:2a:00.0, compute capability: 8.6

validation quality metric 0.7553723127773462
clearing


...

lgbm 1 [0.7443596929686536, 0.7479363322141317, 0.7477357135225149, 0.7463540062258196, 0.7458039271262078]  max  0.7479363322141317  min  0.7443596929686536  avg  0.7464379344114656
starting xgboost


1it [00:37, 37.58s/it]

validation quality metric 0.7477136061635393


2it [01:18, 39.79s/it]

validation quality metric 0.7488975746289146


In [ ]:
out_of_fold_predictions_file = "./outoffoldpredictions.np"
meta_features_file = "./meta_features.np"

In [ ]:
np.save(out_of_fold_predictions_file, out_of_fold_predictions)
np.save(meta_features_file, meta_features)

In [ ]:
meta_model = Ridge()

In [ ]:
# Cross validate ridge
spl = 5
kf = StratifiedKFold(n_splits=spl, shuffle=True)
test_pred_total = np.zeros(len(rawtest))
mse = []
for train_idx, valid_idx in tqdm(kf.split(out_of_fold_predictions,y.round())):
        model = meta_model
        Xt = out_of_fold_predictions[train_idx, :]
        yt = y.iloc[train_idx]
        Xv = out_of_fold_predictions[valid_idx, :]
        yv = y.iloc[valid_idx]
        model.fit(Xt, yt)
        y_pred = model.predict(Xv)
        valid_mse = roc_auc_score(y_true = y.iloc[valid_idx], y_score = y_pred)
        print("valid mse ", valid_mse)
        mse.append(valid_mse)
        test_pred = model.predict(meta_features) / spl
        test_pred_total += test_pred
print("valid mse", mse, " min ", min(mse), " max ", max(mse), " avg ", sum(mse)/len(mse))

In [ ]:
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': rawtest.index,
                       'target': test_pred_total})
output.to_csv('submission.csv', index=False)